In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
import time
import json
import pprint
from random import randint
import math
import datetime

In [37]:
with open('v3_data.txt','r') as f:
    data=f.readlines()
len(data)

44748

In [67]:
with open('v3_result.txt','r') as f:
    data=f.readlines()
len(data)

43135

In [68]:
with open('v3_fail_line.txt','r') as f:
    data=f.readlines()
len(data)

694

In [22]:
res = requests.get('https://auto.8891.com.tw/usedauto-search.html?page=100')
# print(res.text)
js = json.loads(res.text)
js['data']['data']['1']

{'auto_address': '桃園市',
 'auto_brand_en': 'Subaru',
 'auto_brand_id': '43',
 'auto_gas_size': '2.5L',
 'auto_make_date': '<b>2010</b>年',
 'auto_mileage_num': '<b>6.5</b>萬公里',
 'auto_price': '59.8',
 'auto_price_tag': '<b>59.8</b>萬',
 'auto_price_type': '2',
 'auto_tab_name': '手自排',
 'auto_title_all': '《摩鐵汽車廣場》有車在手，遙控啾啾兩聲 你就是帥哥！',
 'auto_year_type': '2010',
 'check_log': '',
 'extra_tag': 0,
 'id': '1437149',
 'is_mobile': '0',
 'is_realPrice': '1',
 'is_red': '0',
 'is_seven': 0,
 'is_vip': '0',
 'item_kind_name_cn': '',
 'item_kind_name_en': 'Legacy Wagon',
 'item_post_date': '2017-06-06 12:07:43',
 'item_renew_date': '<font>53分鐘內更新</font>',
 'item_show_num': '<span class="Red">1962</span>',
 'm_id': '280361',
 'new_item_show_num': 1962,
 'organi': '0',
 'photo': 'https://p2.8891.com.tw/2017/06/06/1/1496721896866792_220_165.jpg',
 'photoList': 'https://p2.8891.com.tw/2017/06/06/1/1496721896866792_300_225.jpg',
 'photoNum': '20',
 'survey_report': '0'}

In [22]:
carDic={'source':'8891'}
info = js['data']['data']['1']
carDic['url']=  'https://auto.8891.com.tw/usedauto-infos-{}.html'.format(info['id'])
carDic['title']= info['auto_title_all']
carDic['brand']= info['auto_brand_en']
carDic['model']= info['item_kind_name_en']
carDic['cc']= info['auto_gas_size']
carDic['transmission']= info['auto_tab_name']
carDic['mileage']= info['auto_mileage_num'].split('<b>')[1].split('</b>')[0]
carDic['years']= info['auto_year_type']
carDic['location']= info['auto_address']
carDic['posttime']= info['item_post_date']
carDic['price']= info['auto_price']
pprint.pprint(carDic)

{'brand': 'Ford',
 'cc': '1.6L',
 'location': '新北市',
 'mileage': '11.0',
 'model': 'Tierra',
 'posttime': '2017-04-18 13:21:02',
 'price': '15.8',
 'source': '8891',
 'title': '05年TIERRA AERO XT )歡迎撥打免付費專線0800090977',
 'transmission': '自排',
 'url': 'https://auto.8891.com.tw/usedauto-infos-1388817.html',
 'years': '2005'}


In [8]:
#generate headers



brand_list = ['AUDI', 'MERCEDES-BENZ', 'BMW', 'FORD', 'HONDA', 'LEXUS', 'MAZDA', 'MITSUBISHI',
              'NISSAN', 'PORSCHE', 'SUZUKI', 'SUBARU', 'TOYOTA', 'VOLVO', 'VOLKSWAGEN']
def gen_headers():
    referers = ['tw.yahoo.com', 'www.google.com', 'http://www.msn.com/zh-tw/', 'http://www.pchome.com.tw/']
    user_agents = ['Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
             'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9',
              'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
              'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)']
    headers = {'user-agent': user_agents[randint(0, len(user_agents) - 1)],
               'referer': referers[randint(0, len(referers) - 1)]}
    return headers


In [9]:
#get total number of cars

def get_car_no(url):
    
    res = requests.get(url)
    js = json.loads(res.text)
    car_no=int(js['data']['total'])
    
    return car_no

url='https://auto.8891.com.tw/usedauto-search.html?page=1'
get_car_no(url)

50570

In [18]:
#get url and first info group(without doors, color, gasoline, equip)
def get_car_urls(brand_list,car_no,headers,file_out):
   
    page_no = math.ceil(car_no / 20)
    
    
    for page in range(1,page_no+1):
        count = 0
        while count<3:
            try:
                url='https://auto.8891.com.tw/usedauto-search.html?page={}'.format(page)
                res = requests.get(url,headers=headers)
                js = json.loads(res.text)
#                 print(js)
                data=""
                
                for i in range(1,len(js['data']['data'].keys())+1):
                    info = js['data']['data'][str(i)] #index OOR solved!
                    if(info['auto_brand_en'].upper() in brand_list):
                        carDic={'source':'8891'}
                        carDic['url'] = 'https://auto.8891.com.tw/usedauto-infos-{}.html'.format(info['id']) 
                        carDic['title']= info['auto_title_all']
                        carDic['brand']= info['auto_brand_en']
                        carDic['model']= info['item_kind_name_en']
                        carDic['cc']= int(float(info['auto_gas_size'].lower().split('l')[0])*1000)
                        carDic['transmission']= info['auto_tab_name']
                        mile_search=re.search(r'[0-9.]+',info['auto_mileage_num'])
                        wen_search = re.search(r'萬',info['auto_mileage_num'])
                        if mile_search:
                            if wen_search:
                                carDic['mileage']= int(float(mile_search.group())*10000)
                            else:
                                carDic['mileage']= int(float(mile_search.group()))
                        else:
                            carDic['mileage']= -1
                        carDic['years']= int(re.search(r'[0-9]+',info['auto_year_type']).group())
                        carDic['location']= info['auto_address']
                        date_str = info['item_post_date']
                        carDic['posttime']= int(datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S").timestamp())
                        carDic['price']= float(info['auto_price'])
                        data += "{}|{}|{}\n".format(carDic['url'],carDic['posttime'],json.dumps(carDic, ensure_ascii=False))
                    else:
                        continue #jump over the car where its brand is not in our brand_list
                with open(file_out,'a') as f:
                    f.write(data)
                with open('progress.csv','w') as f:
                    f.write('{}\n'.format(page))
                break # after succefully write data, break while
            except Exception as e:
                count+=1
                if count==3:
                    message= 'fail page:{},error:{}'.format(page, e)
                    print(message)
#                     with open('debug.txt','a') as f:
#                         f.write('{},{}'.format(info['auto_gas_size'],info['auto_mileage_num']))
                
            finally:
                time.sleep(0.2)
headers=gen_headers()
get_car_urls(brand_list,50570,headers,'v3_data.txt')